<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [4]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [5]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.21.2
Pandas:  1.3.3


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [7]:
# wrong:
bikes = pd.read_table('../data/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('../data/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('../data/bikeshare.csv', header = 0)
print(bikes.head())

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...

  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     

  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                        

?ANSWER:

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [8]:
#ANSWER:
bikes = pd.read_csv('../data/bikeshare.csv', delimiter = ',', header = 0)
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [9]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [10]:
df = pd.read_excel('../data/Iris.xls')
df.head()

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [11]:
#ANSWER
df = pd.read_excel('../data/Iris.xls')
df.head()

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [12]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-11-23T07:17:29.438Z,56.3222,-148.8598,10,4.10,mb,NaN,140,2.5060,0.56,...,2021-11-23T07:54:56.029Z,"252 km SE of Chiniak, Alaska",earthquake,3.60,2.00,0.166,12,reviewed,us,us
1,2021-11-23T07:16:01.640Z,17.9920,-66.8420,17,2.64,md,21.0,136,0.0409,0.14,...,2021-11-23T07:40:37.324Z,"2 km W of Indios, Puerto Rico",earthquake,0.36,0.26,0.100,10,reviewed,pr,pr


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [13]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

# Load 
df = pd.read_html(url)
df = df[0].dropna(axis=0, thresh=4)
print(type(df))
df.head()


<class 'pandas.core.frame.DataFrame'>


,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Almena State Bank,Almena,KS,15426,Equity Bank,"October 23, 2020",10538
1,First City Bank of Florida,Fort Walton Beach,FL,16748,"United Fidelity Bank, fsb","October 16, 2020",10537
2,The First State Bank,Barboursville,WV,14361,"MVB Bank, Inc.","April 3, 2020",10536
3,Ericson State Bank,Ericson,NE,18265,Farmers and Merchants Bank,"February 14, 2020",10535
4,City National Bank of New Jersey,Newark,NJ,21111,Industrial Bank,"November 1, 2019",10534


#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

In [14]:
url = 'http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/'

# Load the XML file into a data frame
df = pd.read_html(url)[0]
df.head()

,Unnamed: 0,ID,String,Description,Type,Comment,Link1,Link2
0,0,id_a_f_3,!Susie (http://www.sync2it.com/susie),Sync2It bookmark management & clustering engine,C R,None,http://www.sync2it.com,None
1,1,id_a_f_6,<a href='http://www.unchaos.com/'> UnChaos </a...,UnCHAOS search robot,R,Site is dead,http://www.unchaos.com/,None
2,2,id_a_f_7,<a href='http://www.unchaos.com/'> UnChaos Bot...,UnCHAOS search robot,R,Site is dead,http://www.unchaos.com/,None


#### Importing JSON Files from the Web
https://www.dataquest.io/blog/python-json-tutorial/

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [15]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [16]:
#ANSWER
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [17]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [18]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [19]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [20]:
#ANSWER
print(bikes['season'])
print(bikes[['season']])

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64
       season
0           1
1           1
2           1
3           1
4           1
...       ...
17374       1
17375       1
17376       1
17377       1
17378       1

[17379 rows x 1 columns]


How would we use object notation to show the first 4 rows of `atemp`?

In [21]:
#ANSWER
#N.B: Call atemp as an object, don't use .head()
atempt = bikes['atemp']
atempt[:4]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [22]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [23]:
#ANSWER
bikes.iloc[0]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed            0.0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [24]:
#ANSWER
bikes.iloc[1]

instant                2
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     1
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.22
atemp             0.2727
hum                  0.8
windspeed            0.0
casual                 8
registered            32
cnt                   40
Name: 1, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [25]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [26]:
#ANSWER
bikes.isnull().sum().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [ ]:
#ANSWER


What is the Numpy object `nan` used for? (Write a descriptive answer.)

Write (and verify) a function that performs simple division with built-in handling of the edge case (i.e. return a nan value instead of just producing an error):

In [27]:
#ANSWER
print(2.3/np.nan)

nan


Apply the Pandas `isna` function to the following data objects:

In [30]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [ ]:
#ANSWER


In [31]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [32]:
#ANSWER
s = pd.Series([x,y])
s.isna()

0    False
1     True
dtype: bool

How is the pandas I/O parameter `na_values` used?

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [33]:
#ANSWER
bikes['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [34]:
#ANSWER
bikes['dteday'] = pd.to_datetime(bikes['dteday'])
print('instant range is from ', bikes.instant.min(), ' to ', bikes.instant.max(), ' is ', bikes.instant.max() - bikes.instant.min())
print('dteday range is from ', bikes.dteday.min(), ' to ', bikes.dteday.max(), ' is ', bikes['dteday'].max() - bikes['dteday'].min())
print('windspeed range is from ', bikes.windspeed.min(), ' to ', bikes.windspeed.max(), ' is ', bikes.instant.max() - bikes.windspeed.min())

instant range is from  1  to  17379  is  17378
dteday range is from  2011-01-01 00:00:00  to  2012-12-31 00:00:00  is  730 days 00:00:00
windspeed range is from  0.0  to  0.8507  is  17379.0


Compute and print the overall minimum and maximum of the data columns:

In [35]:
#ANSWER
print('Overall minimum of Bikes is', bikes.min())
print('Overall minimum of Bikes is', bikes.max())

Overall minimum of Bikes is instant                         1
dteday        2011-01-01 00:00:00
season                          1
yr                              0
mnth                            1
hr                              0
holiday                         0
weekday                         0
workingday                      0
weathersit                      1
temp                         0.02
atemp                         0.0
hum                           0.0
windspeed                     0.0
casual                          0
registered                      0
cnt                             1
dtype: object
Overall minimum of Bikes is instant                     17379
dteday        2012-12-31 00:00:00
season                          4
yr                              1
mnth                           12
hr                             23
holiday                         1
weekday                         6
workingday                      1
weathersit                      4
temp        

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [36]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [37]:
#ANSWER
print('Percentile of 10th, 25th, 50th, 75th, and 90th ', bikes['atemp'].quantile([0.1, 0.25, 0.5, 0.75, 0.9]))

Percentile of 10th, 25th, 50th, 75th, and 90th  0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [38]:
type(bikes['atemp'])

pandas.core.series.Series

In [39]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
6251,6252,2011-09-23,4,0,9,1,0,5,1,2,0.60,0.5000,1.00,0.0000,2,29,31
6388,6389,2011-09-28,4,0,9,18,0,3,1,3,0.66,0.6061,0.83,0.2537,65,459,524
4567,4568,2011-07-14,3,0,7,5,0,4,1,1,0.60,0.6212,0.56,0.2239,6,22,28
11368,11369,2012-04-23,2,1,4,21,0,1,1,2,0.32,0.3030,0.81,0.2239,7,145,152
12674,12675,2012-06-17,2,1,6,7,0,0,0,1,0.52,0.5000,0.72,0.0896,18,30,48


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [40]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)   

What is `atemp_level`?

In [ ]:
#ANSWER


Here is a random sample of `atemp_level`:

In [41]:
atemp_level.sample(5)          

10928    (0.25, 0.5]
1507     (0.25, 0.5]
13441    (0.5, 0.75]
11185    (0.25, 0.5]
4445     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [42]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

16775    mild
1191     cool
3428     warm
2964     warm
9736     mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [43]:
#ANSWER
bikes['atemp_level'] = atemp_level
len(bikes[bikes['season'] == 2])

4409

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [44]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [45]:
# Cleaning Data
print('content of a: ',a)
print('shape of a: ,', a.shape)

print('content of b: ',b)
print('shape of b: ,', b.shape)

print('content of c: ',c)
print('shape of c: ,', c.shape)

print('content of d: ',d)
print('shape of d: ,', d.shape)

print('content of e: ',e)
print('shape of e: ,', e.shape)

content of a:  [0. 0. 0.]
shape of a: , (3,)
content of b:  [[1. 1. 1.]]
shape of b: , (1, 3)
content of c:  [[[1 9 6 9]
  [4 7 2 2]
  [1 5 9 9]]

 [[5 3 7 7]
  [2 1 3 7]
  [3 9 6 5]]]
shape of c: , (2, 3, 4)
content of d:  [0 1 2 3]
shape of d: , (4,)
content of e:  [[1 2 3 4]
 [5 6 7 8]]
shape of e: , (2, 4)


## Load Data

Load rock.csv and clean the dataset.

In [47]:
#ANSWER


df = pd.read_csv('../data/rock.csv')
df.columns = df.columns.str.replace('?','').str.replace('*', '').str.lower().str.replace(' ', '_')
df.head()


/var/folders/vp/k70yvwj57vd00y1yn6zl3mkr0000gn/T/ipykernel_17539/2820638281.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('?','').str.replace('*', '').str.lower().str.replace(' ', '_')


,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [48]:
df

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


## Check Column Names

Check column names and clean.

In [49]:
#ANSWER/
df.columns = df.columns.str.replace('?', '').str.replace('*', '').str.lower().str.replace(' ', '_')

/var/folders/vp/k70yvwj57vd00y1yn6zl3mkr0000gn/T/ipykernel_17539/2061106015.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('?', '').str.replace('*', '').str.lower().str.replace(' ', '_')


In [50]:
df.columns

Index(['song_clean', 'artist_clean', 'release_year', 'combined', 'first',
       'year', 'playcount', 'fg'],
      dtype='object')

In [35]:
df.head()

,song clean,artist clean,release year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [51]:
#ANSWER
df.release_year.fillna(0, inplace = True)
df.release_year

0       1982
1          0
2       1981
3       1980
4       1975
        ... 
2225       0
2226    1981
2227    1975
2228    1983
2229    1973
Name: release_year, Length: 2230, dtype: object

## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [52]:
#ANSWER
df.dtypes
df['release_year'] = df['release_year'].replace('SONGFACTS.COM', '0')
df["release_year"] = df['release_year'].astype('int64')
df.dtypes

song_clean      object
artist_clean    object
release_year     int64
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

In [53]:
df.release_year.astype('int64')

0       1982
1          0
2       1981
3       1980
4       1975
        ... 
2225       0
2226    1981
2227    1975
2228    1983
2229    1973
Name: release_year, Length: 2230, dtype: int64

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [54]:
#ANSWER
df.describe()

,release_year,first,year,playcount,fg
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1465.331390,1.0,0.741256,16.872646,15.048430
std,867.196161,0.0,0.438043,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [55]:
#ANSWER
df[df.release_year<1970]

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,0,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,0,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,0,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,0,Let's Get It Up by AC/DC,1,0,4,0
...,...,...,...,...,...,...,...,...
2216,"I'm Bad, I'm Nationwide",ZZ Top,0,"I'm Bad, I'm Nationwide by ZZ Top",1,0,10,0
2218,Just Got Paid,ZZ Top,0,Just Got Paid by ZZ Top,1,0,2,0
2221,My Head's In Mississippi,ZZ Top,0,My Head's In Mississippi by ZZ Top,1,0,1,0
2222,Party On The Patio,ZZ Top,0,Party On The Patio by ZZ Top,1,0,14,0


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [56]:
#ANSWER
def convert_to_float(df):
    df1 = df.apply(pd.to_numeric, errors = 'coerce')
    return df1

## Apply these functions to your dataset

In [57]:
#ANSWER
df1 = convert_to_float(df)
df1

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,NaN,NaN,1982,NaN,1,1,82,82
1,NaN,NaN,0,NaN,1,0,3,0
2,NaN,NaN,1981,NaN,1,1,85,85
3,NaN,NaN,1980,NaN,1,1,18,18
4,NaN,NaN,1975,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,0,NaN,1,0,1,0
2226,NaN,NaN,1981,NaN,1,1,32,32
2227,NaN,NaN,1975,NaN,1,1,109,109
2228,NaN,NaN,1983,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [58]:
#ANSWER
df1.describe()


,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
count,2.000000,0.0,2230.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1465.331390,NaN,1.0,0.741256,16.872646,15.048430
std,1367.544515,NaN,867.196161,NaN,0.0,0.438043,25.302972,25.288366
min,45.000000,NaN,0.000000,NaN,1.0,0.000000,0.000000,0.000000
25%,528.500000,NaN,0.000000,NaN,1.0,0.000000,1.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



